# Run SnpEff

Genetic variant annotation and effect prediction toolbox. It annotates and predicts the effects of variants on genes (such as amino acid changes). 

### Run VCF Through SnpEff

Requires: 
- SnpEff: http://snpeff.sourceforge.net/
- Reference (downloaded automoatically) GRCh37.74 to match vcf files

### Create VCF from distinct kaviar variants

For each distinct variant found in ISB's Kaivar table, a vcf file will be created and run through SnpEff. 

### Connect to Impala

For HDFS connection, update user argument. 

In [1]:
import ibis
import os

# connect to impala with ibis
hdfs_port = os.environ.get('glados20', 50070)
hdfs = ibis.hdfs_connect(host='glados20', port=hdfs_port, user='selasady')
con = ibis.impala.connect(host='glados19', port=21050, timeout=120)

# enable interactive mode
ibis.options.interactive = True

### Download Distinct Variant Table

This pipeline begins with the table of global distcint variants in impala. Here we are downloading them into an ibis table object to output to vcf and run through snpeff for predicting coding consequences. 

In [2]:
# create ibis object from distinct vars table
#distinct_vars = con.table('global_distinct', database='p7_ref_grch37') 

## ATTN: 
##  using temp table until this table is ready
distinct_vars = con.table('distinct_test', database='users_selasady') 

In [3]:
# limit table to just the columns we need to output to vcf
distinct_df = distinct_vars['chrom', 'pos', 'ref', 'alt']

In [4]:
# download table from ibis table connection object to local memory
distinct_df = distinct_df.execute()

In [5]:
# add varaint ID field for vcf format and downstream matching
distinct_df['ID'] = distinct_df[['chrom', 'pos', 'ref', 'alt']].apply(lambda row: ':'.join(map(str, row)), axis=1)

### Output Distinct Variants as VCF File

In [6]:
import time
import pandas as pd

# disable extraneous pandas warning
pd.options.mode.chained_assignment = None

# enter desired outfile name
vcf_out = 'distinct_vars.vcf'

# create vcf header
def create_header(outfile_name):
   # create vcf header
    lines=[]
    lines.append('##fileformat=VCFv4.0')
    lines.append('##fileDate='+ time.strftime("%y%m%d"))
    lines.append('##reference=grch37 v.74 \n')
    header = '\n'.join(lines)
    out = open(outfile_name, 'wb')
    out.write(header)
    out.close()
               
# create vcf body and append to file with header
def impala_to_vcf(input_df, outfile_name):
    # these columns are output to vcf file
    df = input_df[['chrom', 'pos', 'ID', 'ref', 'alt']]
    # add blank columns for vcf format and format col names
    df['QUAL'] = '30'
    df['FILTER'] = 'PASS'
    df['INFO'] = '.'
    df['FORMAT'] = 'GT'
    df['GT'] = '0/1'
    df.columns = [x.upper() for x in df.columns]
    df=df.rename(columns = {'CHROM':'#CHROM'})
    # order chromosomes to match ref fastas
    chroms = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y', 'M', 'MT']
    df['#CHROM'] = df['#CHROM'].astype("category")
    df['#CHROM'].cat.set_categories(chroms, inplace=True)
    # sort file by chrom then pos
    df = df.sort(['#CHROM', 'POS'])
    # write to file for conversion to vcf
    df.to_csv(outfile_name, header=True, encoding='utf-8', sep="\t", index=False, mode='a')
              
create_header(vcf_out)
impala_to_vcf(distinct_df, vcf_out)

### Run VCF File through SnpEff to predict coding consequences

Running SnpEff on output vcf file: 
    - -Xmx16g : specify amount of java memory 
    - -t : multi-threaded mode, cannot spedify threads, determines automatically
    - -v : verbose mode
    - GRCh37.74 : specifies reference database to use, matched with VCF files

In [8]:
# command to run snpeff
!java -Xmx16g -jar /Users/selasady/tools/snpEff/snpEff.jar -t -v -noStats GRCh37.74 distinct_vars.vcf > distinct_snpeff.vcf

00:00:00.000	SnpEff version SnpEff 4.1a (build 2015-01-14), by Pablo Cingolani
00:00:00.013	Command: 'ann'
00:00:00.028	Reading configuration file 'snpEff.config'. Genome: 'GRCh37.74'
00:00:00.317	done
00:00:00.317	Reading database for genome version 'GRCh37.74' from file '/Users/selasady/tools/snpEff/./data/GRCh37.74/snpEffectPredictor.bin' (this might take a while)
00:00:20.904	done
00:00:20.937	Reading NextProt database from file '/Users/selasady/tools/snpEff/./data/GRCh37.74/nextProt.bin'
00:00:22.067	NextProt database: 550248 markers loaded.
00:00:22.067	Adding transcript info to NextProt markers.
00:00:22.580	NextProt database: 742952 markers added.
00:00:22.580		Loading PWMs from : /Users/selasady/tools/snpEff/./data/GRCh37.74/pwms.bin
00:00:22.603		Loading Motifs from file '/Users/selasady/tools/snpEff/./data/GRCh37.74/motif.bin'
00:00:23.246		Motif database: 284122 markers loaded.
00:00:23.247	Building interval forest
00:00:35.300	done.
00:00:35.300	Genome stats :
#-----------

### Output SnpEff effects as tsv file, one effect per line

Convert the snpeff from vcf format to tsv, with one line per effect, instead of grouped with multiple effects for each position. 

NOTE: vcfEffOnePerLine.pl does not work properly in older verions of SnpEff, make sure you have downloaded the latest version. The perl script is containted within the /snpeff/scripts directory that comes with snpeff. 

In [15]:
cat distinct_snpeff.vcf \
    | /Users/selasady/tools/snpEff/scripts/vcfEffOnePerLine.pl \
    | java -jar /Users/selasady/tools/snpEff/SnpSift.jar extractFields \
    - CHROM POS REF ALT "ANN[*].GENE" "ANN[*].GENEID" "ANN[*].EFFECT" "ANN[*].IMPACT" \
    "ANN[*].FEATURE" "ANN[*].FEATUREID" "ANN[*].BIOTYPE" "ANN[*].RANK" \
    "ANN[*].HGVS_C" "ANN[*].HGVS_P" > distinct.tsv 

### Remove Header and convert to CSV

To prepare for upload to HDFS and conversion into impala table, the header will be removed so we can specify our own column names in the schema below and parse. 

In [18]:
!sed '1d' distinct.tsv | tr '\t' ',' > distinct_snpeff.csv

## Save Results as an Impala Table

### Upload Results to HDFS

The CSV file is uploaded to HDFS, in a directory called 'snpeff'. 

In [34]:
# create directory
hdfs.mkdir('/user/selasady/snpeff')

# upload file
path = '/user/selasady/snpeff'
file_name = 'distinct_snpeff.csv'
admix_file = path + '/' + file_name

# upload admix file
hdfs.put(path, file_name, verbose=True)

'/user/selasady/snpeff/distinct_snpeff.csv'

### Convert TSV into Ibis Object

Once the TSV file is in HDFS, we can use ibis to convert it into an impala table by pairing the file with a schema. 

In [35]:
# define table schema for tsv file
schema = ibis.schema([
    ('chrom', 'string'), 
    ('pos', 'int32'),
    ('ref', 'string'),
    ('alt', 'string'),
    ('gene', 'string'),
    ('gene_id', 'string'),
    ('effect', 'string'),
    ('impact', 'string'),
    ('feature', 'string'),
    ('feature_id', 'string'),
    ('biotype', 'string'),
    ('rank', 'int32'),
    ('hgvs_c', 'string'),
    ('hgvs_p', 'string')
])

# create ibis object from  tsv
snpeff = con.delimited_file(path, schema)

In [36]:
print snpeff.limit(5)

  chrom     pos ref alt           gene                          gene_id  \
0     1   64548   A   C          OR4F5                  ENSG00000186092   
1     1   64548   A   C        OR4G11P                  ENSG00000240361   
2     1   64548   A   C  OR4G11P-OR4F5  ENSG00000240361-ENSG00000186092   
3     1  149954   G   C  RP11-34P13.13                  ENSG00000241860   
4     1  149954   G   C  RP11-34P13.13                  ENSG00000241860   

                    effect    impact            feature  \
0    upstream_gene_variant  MODIFIER         transcript   
1  downstream_gene_variant  MODIFIER         transcript   
2        intergenic_region  MODIFIER  intergenic_region   
3    upstream_gene_variant  MODIFIER         transcript   
4    upstream_gene_variant  MODIFIER         transcript   

                        feature_id                 biotype  rank     hgvs_c  \
0                  ENST00000335137          protein_coding    -1    c.-1A>C   
1                  ENST00000492842  

### Create impala table

Once the TSV file is paired with a schema, we can save it as a table in impala. 

In [38]:
con.create_table('coding_consequences', snpeff, database='p7_ref_grch37')